In [ ]:
import pandas as pd

def compute_daily_station_net_change(input_csv, output_csv=None):
    """
    Pour un CSV de trajets (contenant au minimum 'started_at', 'start_station_id', 'end_station_id'),
    calcule pour chaque station par jour :
      - Le nombre d'arrivées (où la station apparaît en tant que end_station_id)
      - Le nombre de départs (où la station apparaît en tant que start_station_id)
      - La variation nette : (arrivées - départs)
    
    Le DataFrame final est pivoté, avec :
      - Une ligne par date (extraction de la date depuis 'started_at')
      - Une colonne par station (identifiant)
      - Les valeurs étant la variation nette des vélos pour la station à cette date.

    Arguments:
    ----------
    input_csv : str
        Chemin du CSV de trajets.
    output_csv : str ou None
        Chemin de sauvegarde du résultat (optionnel).

    Retourne:
    --------
    df_pivot : DataFrame
        Tableau final au format demandé.
    """
    # 1. Chargement du CSV
    df = pd.read_csv(input_csv, low_memory=False)
    
    # 2. Conversion de la date à partir de 'started_at' (on prend la date de départ)
    df['date'] = pd.to_datetime(df['started_at'], errors='coerce').dt.date

    # 3. Calcul des arrivées par station et par jour
    arrivals = df.groupby(['date', 'end_station_id']).size().reset_index(name='arrivals')
    # Renommage pour harmoniser avec les départs
    arrivals = arrivals.rename(columns={'end_station_id': 'station_id'})
    
    # 4. Calcul des départs par station et par jour
    departures = df.groupby(['date', 'start_station_id']).size().reset_index(name='departures')
    departures = departures.rename(columns={'start_station_id': 'station_id'})
    
    # 5. Fusionner les arrivées et départs pour obtenir les deux valeurs par station et par jour
    df_merged = pd.merge(arrivals, departures, on=['date', 'station_id'], how='outer')
    df_merged['arrivals'] = df_merged['arrivals'].fillna(0)
    df_merged['departures'] = df_merged['departures'].fillna(0)
    
    # 6. Calcul de la variation nette : (arrivées - départs)
    df_merged['net_change'] = df_merged['arrivals'] - df_merged['departures']
    
    # 7. Pivot du DataFrame pour avoir :
    #    - Index : date
    #    - Colonnes : station_id
    #    - Valeurs : net_change
    df_pivot = df_merged.pivot(index='date', columns='station_id', values='net_change')
    df_pivot = df_pivot.fillna(0).reset_index()
    
    # 8. (Optionnel) Sauvegarde du résultat dans un CSV
    if output_csv is not None:
        df_pivot.to_csv(output_csv, index=False)
    
    return df_pivot

# =========================
# Exemple d'utilisation
# =========================
if __name__ == "__main__":
    input_file = "202004-divvy-tripdata.csv"  # Remplacez par le chemin complet si nécessaire
    output_file = "daily_net_change.csv"       # Chemin de sortie désiré
    
    df_result = compute_daily_station_net_change(input_file, output_file)
    print("Aperçu du tableau final :")
    print(df_result.head())

In [ ]:
def compute_daily_station_net_change(input_csv):
    """
    Pour un CSV de trajets (contenant au moins 'started_at', 'start_station_id', 'end_station_id'),
    calcule pour chaque station par jour :
      - Le nombre d'arrivées (où la station apparaît en tant que end_station_id)
      - Le nombre de départs (où la station apparaît en tant que start_station_id)
      - La variation nette : (arrivées - départs)
    
    Retourne:
      df_pivot : DataFrame au format demandé, avec une ligne par date et une colonne par station.
    """
    # 1. Chargement du CSV
    df = pd.read_csv(input_csv, low_memory=False)
    
    # 2. Conversion de 'started_at' en date (on utilise la date de départ)
    df['date'] = pd.to_datetime(df['started_at'], errors='coerce').dt.date
    
    # 3. Calcul des arrivées par station et par jour
    arrivals = df.groupby(['date', 'end_station_id']).size().reset_index(name='arrivals')
    arrivals = arrivals.rename(columns={'end_station_id': 'station_id'})
    
    # 4. Calcul des départs par station et par jour
    departures = df.groupby(['date', 'start_station_id']).size().reset_index(name='departures')
    departures = departures.rename(columns={'start_station_id': 'station_id'})
    
    # 5. Fusionner les arrivées et départs
    df_merged = pd.merge(arrivals, departures, on=['date', 'station_id'], how='outer')
    df_merged['arrivals'] = df_merged['arrivals'].fillna(0)
    df_merged['departures'] = df_merged['departures'].fillna(0)
    
    # 6. Calcul de la variation nette (arrivées - départs)
    df_merged['net_change'] = df_merged['arrivals'] - df_merged['departures']
    
    # 7. Pivot du DataFrame pour obtenir le format souhaité :
    #    - Index : date
    #    - Colonnes : station_id
    #    - Valeurs : net_change
    df_pivot = df_merged.pivot(index='date', columns='station_id', values='net_change')
    df_pivot = df_pivot.fillna(0).reset_index()
    
    return df_pivot
